In [ ]:
!pip install keras
!pip install keras.utils

  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=b1b5d0b7e0557a93665603908ff2a42901a86f1bad1ce7577834328fed711da9
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [718 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.lau

In [ ]:
!pip3 install JPype1-py3

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Add
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab/agro/data/agro_clean.csv')

#Load Tokenized Data

In [ ]:
okt = np.load('/content/drive/MyDrive/Colab/agro/data/okt_tokens.npy', allow_pickle=True)
okt_stemmed = np.load('/content/drive/MyDrive/Colab/agro/data/okt_tokens_stemmed.npy', allow_pickle=True)
df['okt_tokenized'] = okt
print(okt)
print(okt.shape)

[list(['남녀', '중', '우월하다', '성별', '통계학', '적', '증명', '천조국', '예능', 'ㄷ'])
 list(['천재견', '새끼', '건', '벌어지다', '일'])
 list(['계약', '직', '회사', '들어가다', '벌어지다', '일']) ...
 list(['기생충', '그녀', '형수', '님', '되다', '얼마나', '무섭다', '보여주다', '영화'])
 list(['천재', '흑인', '백인', '엘리트', '차다', '교육', '하', '영화'])
 list(['창고에', '버려진', '싸움의', '천재가', '국정원', '급', '요원이', '되면', '벌어지는', '일'])]
(1881,)


In [ ]:
# mecab = Mecab()

# df['mecab_tokenized'] = df['유튜브 제목'].apply(mecab.morphs)
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
# df['mecab_tokenized'] = df['mecab_tokenized'].apply(lambda x: [token for token in x if token not in stopwords])

In [ ]:
df.head(2)

,영화 제목,썸네일,유튜브 제목,어그로표현 있음,내용,자극,의문,THUMB,내용있는어그로,video_id,자극+의문,char,okt_tokenized
0,"100인,인간을말하다",https://i.ytimg.com/vi/1FhZxcKYmE0/hqdefault.jpg,남녀 중 우월한 성별을 통계학적으로 증명하는 천조국 예능 ㄷ,1,0,1,0,1,0,1FhZxcKYmE0,1,9,"[남녀, 중, 우월하다, 성별, 통계학, 적, 증명, 천조국, 예능, ㄷ]"
1,101달마시안,https://i.ytimg.com/vi/ntplV87tOMw/hqdefault.jpg,천재견의 새끼들을 건들면 벌어지는 일,1,1,1,1,0,1,ntplV87tOMw,2,5,"[천재견, 새끼, 건, 벌어지다, 일]"


In [ ]:
np.sum(df['어그로표현 있음'])

1323

In [ ]:
X = okt
# X = np.array(df['mecab_tokenized'])
# y = np.array( df['내용있는어그로'])
# y = np.array( df['내용'])
# y = np.array( df['의문'])
# y = np.array( df['자극'])
# y = np.array( df['주내용 (493)'])
# y = np.array( df['부분내용(924)'])
# y = np.array( df['내용없음(452)'])
y = np.array( df['어그로표현 있음'])

In [ ]:
print(sum(y))

1323


#Split Data

In [ ]:
from collections import Counter
RANDOM_SEED = 42
TEST_SPLIT = 0.2

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

# summarize class distribution before SMOTE
print("Before oversampling: ",Counter(y_train))

Before oversampling:  Counter({1: 1048, 0: 456})


#Keras Embedding

In [ ]:
'''
if you fit your tokenizer on whole dataset you are somehow biasing your model. 
For a good evaluation of your model, you have to take in account the UNK tokens. 
'''

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

threshold = 2
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 2

print('vocab_size', vocab_size)
print('rare_cnt', rare_cnt)
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

characters = [len(l) for l in x_train]
print('최대 길이 : ', max(characters))
print('max index: ', characters.index(max(characters)))
print('가장 긴 제목: ', x_train[86])
print('가장많이 나온 토큰 순위: ', Counter(tokenizer.word_counts))
print('평균 길이: ',sum(map(len, x_train))/len(x_train))

vocab_size 1441
rare_cnt 1794
단어 집합(vocabulary)의 크기 : 3233
등장 빈도가 1번 이하인 희귀 단어의 수: 1794
단어 집합에서 희귀 단어의 비율: 55.49025672749768
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 15.141796083727211
최대 길이 :  21
max index:  827
가장 긴 제목:  ['세', '납치', '돼다', '그', '방', '임신', '출산', '되다', '여자', '소개', '추천']
가장많이 나온 토큰 순위:  Counter({'영화': 289, '일': 206, '되다': 188, '벌어지다': 146, '?': 126, '남자': 111, '버리다': 96, '여자': 91, '이유': 84, '보다': 80, '있다': 71, '사람': 62, '!': 61, '인간': 61, '년': 60, 'ㄷ': 59, '미치다': 55, '방법': 53, '적': 52, '천재': 51, '수': 51, 'ㅋ': 50, '넷플릭스': 49, '한국': 48, '만들다': 47, '없다': 46, '생기다': 46, '최후': 45, '소녀': 43, '하이': 41, '틴': 41, '미래': 39, '사랑': 38, '충격': 38, '지구': 34, '위': 34, '실화': 34, '세상': 34, '좀비': 33, '시키다': 32, '못': 31, '명': 30, '초능력': 29, '급': 29, '레전드': 28, '추천': 28, '아이': 28, '인류': 28, '먹다': 28, '죽다': 27, '미국': 27, '정말': 26, '전': 26, '살': 25, '신': 25, '싸이코패스': 25, '집': 25, '이야기': 25, '안되다': 25, '것': 25, '모든': 25, '세계': 24, '당하다': 24, '건드리다': 24, '최강': 24, '갇히다': 23, '위해': 23, '진짜': 23, '인생': 23, '않다'

In [ ]:
'''
fit_on_texts {'my': 1, 'love': 2, 'dog': 3, 'i': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}
texts to sequences [[4, 2, 1, 3], [4, 2, 1, 6], [5, 2, 1, 3], [7, 5, 8, 1, 3, 9, 10]]
'''

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') #oov_token = 'OOV' 
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

max_len = 21

x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(1504, 21)
(377, 21)


In [ ]:
1881*0.01

#SMOTE 

In [ ]:
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

SMOTE = SMOTE()

# fit and apply the transform
x_train, y_train = SMOTE.fit_resample(x_train, y_train)

# summarize class distribution after SMOTE
print("After oversampling: ",Counter(y_train))

After oversampling:  Counter({1: 1048, 0: 1048})


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#Model

In [ ]:
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

# e = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_len, trainable=False)

model = Sequential()
# model.add(e)
model.add(Embedding(vocab_size, 128))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

Adagrad

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, callbacks=[es, mc], batch_size=16, validation_split=0.1)

Epoch 1/10
118/118 [==============================] - 11s 22ms/step - loss: 0.6041 - acc: 0.6776 - val_loss: 0.9367 - val_acc: 0.3905

Epoch 00001: val_acc improved from -inf to 0.39048, saving model to best_model.h5
Epoch 2/10
118/118 [==============================] - 2s 14ms/step - loss: 0.4518 - acc: 0.7969 - val_loss: 0.7205 - val_acc: 0.5952

Epoch 00002: val_acc improved from 0.39048 to 0.59524, saving model to best_model.h5
Epoch 3/10
118/118 [==============================] - 2s 15ms/step - loss: 0.3518 - acc: 0.8473 - val_loss: 0.9329 - val_acc: 0.5905

Epoch 00003: val_acc did not improve from 0.59524
Epoch 4/10
118/118 [==============================] - 2s 15ms/step - loss: 0.2805 - acc: 0.8876 - val_loss: 0.9632 - val_acc: 0.5714

Epoch 00004: val_acc did not improve from 0.59524
Epoch 5/10
118/118 [==============================] - 2s 15ms/step - loss: 0.2269 - acc: 0.9077 - val_loss: 0.9611 - val_acc: 0.6190

Epoch 00005: val_acc improved from 0.59524 to 0.61905, saving 

In [ ]:
from sklearn.metrics import classification_report

y_pred=model.predict(x_test, batch_size=16, verbose=1)
report = classification_report(y_test, y_pred.round(), digits=4)

print(report)


24/24 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0     0.5913    0.6667    0.6267       102
           1     0.8702    0.8291    0.8492       275

    accuracy                         0.7851       377
   macro avg     0.7308    0.7479    0.7379       377
weighted avg     0.7948    0.7851    0.7890       377



bestmodel is in '/content/best_model.h5'


In [ ]:
# model.save('/content/drive/MyDrive/Colab/agro/model')

In [ ]:
# from tensorflow import keras 
# loaded_model = keras.models.load_model('/content/drive/MyDrive/Colab/agro/model')

In [ ]:
어그로 표현 +SMOTE (70%)

             precision    recall  f1-score   support

           0     0.7216    0.6863    0.7035       102
           1     0.8857    0.9018    0.8937       275

    accuracy                         0.8435       377
   macro avg     0.8037    0.7940    0.7986       377
weighted avg     0.8413    0.8435    0.8422       377

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=1441, output_dim=128, input_length=21)) #vocab size, embedding dim, max length
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 128)           184448    
_________________________________________________________________
conv1d (Conv1D)              (None, 21, 64)            24640     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                20512     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 229,633
Trainable params: 229,633
Non-trainable params: 0
________________________________________________

In [ ]:
import tensorflow as tf
epochs = 10
batch_size = 32

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/10
59/59 [==============================] - 24s 10ms/step - loss: 0.6444 - accuracy: 0.6161 - val_loss: 0.7923 - val_accuracy: 0.5048
Epoch 2/10
59/59 [==============================] - 0s 6ms/step - loss: 0.4223 - accuracy: 0.8155 - val_loss: 0.7144 - val_accuracy: 0.6571
Epoch 3/10
59/59 [==============================] - 0s 6ms/step - loss: 0.1961 - accuracy: 0.9353 - val_loss: 0.8588 - val_accuracy: 0.6238
Epoch 4/10
59/59 [==============================] - 0s 6ms/step - loss: 0.0655 - accuracy: 0.9862 - val_loss: 1.4326 - val_accuracy: 0.5619
Epoch 5/10
59/59 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.9963 - val_loss: 1.2794 - val_accuracy: 0.6143
Epoch 6/10
59/59 [==============================] - 0s 5ms/step - loss: 0.0128 - accuracy: 0.9973 - val_loss: 1.8211 - val_accuracy: 0.5476
Epoch 7/10
59/59 [==============================] - 0s 5ms/step - loss: 0.0101 - accuracy: 0.9979 - val_loss: 1.8807 - val_accuracy: 0.5429
Epoch 8/10
59/59 [

In [ ]:
from sklearn.metrics import classification_report

y_pred=model.predict(x_test, batch_size=32, verbose=1)
report = classification_report(y_test, y_pred.round(), digits=4)
print(report)

12/12 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0     0.6200    0.6078    0.6139       102
           1     0.8556    0.8618    0.8587       275

    accuracy                         0.7931       377
   macro avg     0.7378    0.7348    0.7363       377
weighted avg     0.7919    0.7931    0.7925       377



In [ ]:
어그로 표현 +SMOTE
              precision    recall  f1-score   support

           0     0.6774    0.6176    0.6462       102
           1     0.8627    0.8909    0.8766       275

    accuracy                         0.8170       377
   macro avg     0.7700    0.7543    0.7614       377
weighted avg     0.8126    0.8170    0.8142       377